In [208]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as preprocessing 
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Load MNIST Dataset

In [132]:
# Training Data
Train = pd.read_csv('/home/tonzhu/Downloads/mnist_train.csv', header = None)
print(Train.shape)
Train.head(10)

(60000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
Train.describe()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
count,60000.000000,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,4.453933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,2.889270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


In [134]:
Train_data = Train.iloc[:,1:]
Train_label = Train.iloc[:,0]

In [135]:
# Test Data
Test = pd.read_csv('/home/tonzhu/Downloads/mnist_test.csv', header = None)
print(Test.shape)
Test.head(10)

(10000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
Test_data = Test.iloc[:,1:]
Test_label = Test.iloc[:,0]

# Data Preprocessing

In [164]:
# Normalization (min_max, make number between 0 and 1)
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True).fit(Train_data)
X_train_minmax = minmax_scale.transform(Train_data)
X_test_minmax = minmax_scale.transform(Test_data)

# PCA Feature Extraction

In [175]:
# PCA Extraction
pca = PCA(n_components = 200).fit(X_train_minmax)
Train_pca = pca.transform(X_train_minmax)
Test_pca = pca.transform(X_test_minmax)

# Modeling

In [153]:
Train_label = np.array(Train_label)
Test_label = np.array(Test_label)

Logistic Regression

In [176]:
LR = LogisticRegression(solver = 'newton-cg', multi_class = 'multinomial')

In [177]:
LR.fit(Train_pca, Train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [178]:
accuracy = float(np.trace(confusion_matrix(Train_label, LR.predict(Train_pca)))) / float(len(Train_pca))

In [179]:
accuracy 

0.9305666666666667

In [180]:
accuracy_test = float(np.trace(confusion_matrix(Test_label, LR.predict(Test_pca)))) / float(len(Test_pca))

In [181]:
accuracy_test 

0.9258

In [183]:
confusion_matrix(Test_label, LR.predict(Test_pca))

array([[ 956,    0,    2,    4,    0,    8,    5,    4,    1,    0],
       [   0, 1111,    4,    1,    0,    1,    4,    2,   12,    0],
       [   6,    8,  932,   14,    7,    2,   13,    7,   37,    6],
       [   4,    1,   21,  918,    1,   23,    4,   11,   20,    7],
       [   1,    1,    5,    3,  913,    0,   10,    5,   10,   34],
       [  10,    2,    2,   32,    8,  779,   15,    7,   33,    4],
       [   9,    3,    7,    2,    7,   11,  915,    2,    2,    0],
       [   1,    6,   23,    5,    7,    1,    0,  953,    3,   29],
       [   8,    7,    8,   22,    7,   29,   12,   10,  863,    8],
       [  10,    7,    1,    9,   26,    7,    0,   22,    9,  918]])

In [ ]:
# cross validation select parameters
# Radial Kernel
param_grid = { "C" : [0.001, 0.01, 0.1, 1, 10], "gamma" : [0.001, 0.01, 0.1, 1, 10]}
SVM = SVC()
gs = GridSearchCV(estimator=SVM, param_grid=param_grid, scoring='accuracy', cv=10, verbose=1)
gs = gs.fit(Train_pca, Train_label)
print(gs.best_score_)
print(gs.best_params_)

SVM

In [ ]:
# Linear Kernel
param_grid = { "C" : [0.001, 0.01, 0.1, 1, 10]}
SVM = SVC(kernel = 'linear')
gs_linsvm = GridSearchCV(estimator=SVM, param_grid=param_grid, scoring='accuracy', cv=10, verbose=1)
gs_linsvm = gs_linsvm.fit(Train_pca, Train_label)
print(gs_linsvm.best_score_)
print(gs_linsvm.best_params_)

In [ ]:
# Polinomial Kernel
param_grid = { "C" : [0.001, 0.01, 0.1, 1, 10], "degree" : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
SVM = SVC(kernel = 'poly')
gs_polysvm = GridSearchCV(estimator=SVM, param_grid=param_grid, scoring='accuracy', cv=10, verbose=1)
gs_polysvm = gs_polysvm.fit(Train_pca, Train_label)
print(gs_polysvm.best_score_)
print(gs_polysvm.best_params_)

In [193]:
SVM = SVC(C = 0.1, gamma = 0.01, degree = 9)
SVM.fit(Train_pca, Train_label)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=9, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [199]:
confusion_matrix(Train_label, SVM.predict(Train_pca))

array([[   0, 5923,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 6742,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5958,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 6131,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5842,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5421,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5918,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 6265,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5851,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5949,    0,    0,    0,    0,    0,    0,    0,    0]])

In [200]:
accuracy_SVM = float(np.trace(confusion_matrix(Train_label, SVM.predict(Train_pca)))) / float(len(Train_pca))

In [195]:
accuracy_SVM

0.11236666666666667

In [196]:
accuracy_test_SVM = float(np.trace(confusion_matrix(Test_label, LR.predict(Test_pca)))) / float(len(Test_pca))

In [197]:
accuracy_test_SVM

0.9258

In [ ]:
confusion_matrix(Test_label, SVM.predict(Test_pca))

Random Forest

In [201]:
RF = RandomForestClassifier(n_estimators = 500)
RF.fit(Train_pca, Train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [202]:
accuracy_RF = float(np.trace(confusion_matrix(Train_label, RF.predict(Train_pca)))) / float(len(Train_pca))

In [204]:
accuracy_RF

1.0

In [205]:
accuracy_test_RF = float(np.trace(confusion_matrix(Test_label, RF.predict(Test_pca)))) / float(len(Test_pca))

In [206]:
accuracy_test_RF

0.9516

In [ ]:
GBM

In [ ]:
# cross validation select parameters
param_grid = { 'learning_rate': [0.001, 0.01, 0.1], 'max_depth': [3, 6, 10, 15], 'n_estimators': [50, 100, 200]}
gbm = GradientBoostingClassifier()
gs_gbm = GridSearchCV(estimator=gbm, param_grid=param_grid, scoring='accuracy', cv=10, verbose=1)
gs_gbm = gs_gbm.fit(Train_pca, Train_label)

In [ ]:
GBM = GradientBoostingClassifier(learning_rate = 0.005, max_depth = 10, n_estimators = 100)
GBM.fit(Train_pca, Train_label)

In [210]:
accuracy_GBM = float(np.trace(confusion_matrix(Train_label, GBM.predict(Train_pca)))) / float(len(Train_pca))
accuracy_GBM

0.9872333333333333

In [212]:
accuracy_test_GBM = float(np.trace(confusion_matrix(Test_label, GBM.predict(Test_pca)))) / float(len(Test_pca))
accuracy_test_GBM

0.9149